In [2]:
import pandas

In [6]:
feeds = pandas.read_csv("../../data/FeedBaby-data-export-20180304_202755/feeds.csv")

In [11]:
feeds.sample(n=10).head()

,id,Start Time,End Time,Feed Type,Quantity (oz),Quantity (ml or g),Notes,Duration (Minutes),Food Type,Unit,Bottle Type
828,1724,03:32:15 10-25-2017,03:40:15 10-25-2017,Bottle,3.38,100,NaN,8,Liquid,METRIC,Formula
952,1600,01:28:12 10-11-2017,01:45:12 10-11-2017,Bottle,5.92,175,NaN,17,Liquid,METRIC,Formula
1411,1133,07:24:05 08-14-2017,07:33:17 08-14-2017,Bottle,3.38,100,NaN,9,Liquid,METRIC,Formula
1526,1017,19:46:08 08-01-2017,19:50:59 08-01-2017,Bottle,1.69,50,NaN,4,Liquid,METRIC,Formula
1743,799,13:17:26 07-11-2017,13:24:26 07-11-2017,Bottle,2.37,70,NaN,7,Liquid,METRIC,Formula


In [10]:
feeds.describe()


,id,Quantity (oz),Quantity (ml or g),Duration (Minutes)
count,2530.000000,2530.000000,2530.000000,2530.000000
mean,1281.190909,3.702921,109.503953,10.089328
std,736.290294,1.490045,44.078153,16.122993
min,2.000000,0.340000,10.000000,0.000000
25%,644.250000,2.540000,75.000000,8.000000
50%,1280.500000,3.380000,100.000000,9.000000
75%,1919.750000,5.070000,150.000000,10.000000
max,2554.000000,9.300000,275.000000,729.000000
